In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install selenium

In [4]:
import requests
from bs4 import BeautifulSoup
from nltk import word_tokenize
import numpy as np
from time import sleep
from random import randint
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import argparse

In [9]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import numpy as np

# Define constants
OUTPUT_DIR_PATH = 'C:/Users/pktal/OneDrive/Documents/Mouthshut reviews/Unacademy reviews.csv'
URL = 'https://www.mouthshut.com/websites/unacademy-com-reviews-925890850'
NO_PAGES = 9

# Chrome options for disabling notifications while scraping
option = Options()
option.add_argument("--disable-infobars")
option.add_argument("start-maximized")
option.add_argument("--disable-extensions")
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--disable-dev-shm-usage')
option.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 1})

# Set path to ChromeDriver executable
chrome_driver_path = 'C:/Users/pktal/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe'  # Update this path

# Initialize Chrome driver
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=option)

pages = np.arange(1, NO_PAGES + 1)  # Page list

# Prepare the CSV file
with open(OUTPUT_DIR_PATH, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['user_name', 'user_location', 'review_count', 'review_title', 'review_datetime', 'review_views', 'review_content']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for page in pages:
        url = URL + '-page-' + str(page)
        
        driver.get(url)
        time.sleep(3)  # Wait for the page to load

        try:
            for elem in driver.find_elements(By.LINK_TEXT, 'Read More'):  # Clicking 'Read More' buttons
                elem.click()
                time.sleep(0.2)  # Adding a slight delay to prevent too many requests

            # Get the page source and parse with BeautifulSoup
            page_source = driver.page_source
            html_soup = BeautifulSoup(page_source, 'html.parser')
            data_containers = html_soup.find_all('div', class_='row review-article')

            for container in data_containers:
                user_name = container.find('div', class_='user-ms-name').text.strip() if container.find('div', class_='user-ms-name') else 'N/A'
                user_location = container.find('div', class_='usr-addr-text').text.strip() if container.find('div', class_='usr-addr-text') else 'N/A'
                review_count = container.find('div', class_='total-rev-counts').text.strip() if container.find('div', class_='total-rev-counts') else 'N/A'
                review_title = container.find('a', id=lambda x: x and x.startswith('rptreviews_ctl')).text.strip() if container.find('a', id=lambda x: x and x.startswith('rptreviews_ctl')) else 'N/A'
                review_datetime = container.find('span', id=lambda x: x and x.startswith('rptreviews_ctl')).text.strip() if container.find('span', id=lambda x: x and x.startswith('rptreviews_ctl')) else 'N/A'
                review_views = container.find('span', class_='views').text.strip() if container.find('span', class_='views') else 'N/A'
                review_content = container.find('div', class_='more reviewdata').text.strip() if container.find('div', class_='more reviewdata') else 'N/A'

                writer.writerow({
                    'user_name': user_name,
                    'user_location': user_location,
                    'review_count': review_count,
                    'review_title': review_title,
                    'review_datetime': review_datetime,
                    'review_views': review_views,
                    'review_content': review_content
                })

        except Exception as e:
            print(f"Error occurred: {e}")

driver.quit()
